In [1]:
import sys
sys.path.append('/home/jovyan/data/data_CSV')
from extraction import extract_data
from transformation import transform_data
from loading import load_data
from schema import create_schema

file_path = '/home/jovyan/data/data_CSV/online_retail.csv'
db_url = 'postgresql://myuser:01144873329@db:5432/mydatabase' 

data = extract_data(file_path)

print(data.head())




  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

           InvoiceDate  UnitPrice  CustomerID         Country  
0  2010-12-01 08:26:00       2.55     17850.0  United Kingdom  
1  2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
2  2010-12-01 08:26:00       2.75     17850.0  United Kingdom  
3  2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
4  2010-12-01 08:26:00       3.39     17850.0  United Kingdom  


In [2]:
print("Number of not NULL before:\n")
print(data.info())




Number of not NULL before:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB
None


In [3]:

data.columns



Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [4]:
data.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [5]:
print("Number of duplicated before:", data.duplicated().sum())

Number of duplicated before: 5268


In [6]:
transformed_data,datetime_dim_data, customer_dim_data, product_dim_data, fact_table_data = transform_data(data)

In [7]:
transformed_data.describe()

,Quantity,CustomerID
count,401564.000000,401564.000000
mean,13.509769,15281.266797
std,249.442715,1713.978947
min,1.000000,12346.000000
25%,2.000000,13939.000000
50%,6.000000,15145.000000
75%,12.000000,16788.000000
max,80995.000000,18287.000000


In [8]:
print("Number of not NULL After:\n")
print(transformed_data.info())


Number of not NULL After:

<class 'pandas.core.frame.DataFrame'>
Index: 401564 entries, 0 to 541908
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   InvoiceNo         401564 non-null  object
 1   StockCode         401564 non-null  object
 2   Description       401564 non-null  object
 3   Quantity          401564 non-null  int64 
 4   UnitPrice         401564 non-null  object
 5   CustomerID        401564 non-null  int64 
 6   Country           401564 non-null  object
 7   InvoiceDate_Date  401564 non-null  object
 8   InvoiceDate_Time  401564 non-null  object
dtypes: int64(2), object(7)
memory usage: 30.6+ MB
None


In [9]:
print("Number of duplicated after:", transformed_data.duplicated().sum())

Number of duplicated after: 0


In [10]:
create_schema(db_url)

Schema created successfully.


In [11]:


load_data(db_url, datetime_dim_data, customer_dim_data, product_dim_data, fact_table_data)

Data successfully loaded into PostgreSQL.


In [12]:
import psycopg2
conn = psycopg2.connect(db_url)
cur = conn.cursor()
cur.execute("SELECT * FROM datetime_dim LIMIT 20;")
rows = cur.fetchall()
for row in rows:
    print(row)

(1, datetime.date(2010, 12, 1), datetime.time(8, 26))
(2, datetime.date(2010, 12, 1), datetime.time(8, 28))
(3, datetime.date(2010, 12, 1), datetime.time(8, 34))
(4, datetime.date(2010, 12, 1), datetime.time(8, 35))
(5, datetime.date(2010, 12, 1), datetime.time(8, 45))
(6, datetime.date(2010, 12, 1), datetime.time(9, 0))
(7, datetime.date(2010, 12, 1), datetime.time(9, 1))
(8, datetime.date(2010, 12, 1), datetime.time(9, 2))
(9, datetime.date(2010, 12, 1), datetime.time(9, 9))
(10, datetime.date(2010, 12, 1), datetime.time(9, 32))
(11, datetime.date(2010, 12, 1), datetime.time(9, 34))
(12, datetime.date(2010, 12, 1), datetime.time(9, 37))
(13, datetime.date(2010, 12, 1), datetime.time(9, 41))
(14, datetime.date(2010, 12, 1), datetime.time(9, 45))
(15, datetime.date(2010, 12, 1), datetime.time(9, 49))
(16, datetime.date(2010, 12, 1), datetime.time(9, 53))
(17, datetime.date(2010, 12, 1), datetime.time(9, 56))
(18, datetime.date(2010, 12, 1), datetime.time(9, 57))
(19, datetime.date(2010

In [13]:
import psycopg2
conn = psycopg2.connect(db_url)
cur = conn.cursor()
cur.execute("SELECT * FROM fact_table LIMIT 20;")
rows = cur.fetchall()
for row in rows:
    print(row)

(1, '536365', '85123A', 6, Decimal('2.55'), 1, 17850)
(2, '536365', '71053', 6, Decimal('3.39'), 1, 17850)
(3, '536365', '84406B', 8, Decimal('2.75'), 1, 17850)
(4, '536365', '84029G', 6, Decimal('3.39'), 1, 17850)
(5, '536365', '84029E', 6, Decimal('3.39'), 1, 17850)
(6, '536365', '22752', 2, Decimal('7.65'), 1, 17850)
(7, '536365', '21730', 6, Decimal('4.25'), 1, 17850)
(8, '536366', '22633', 6, Decimal('1.85'), 2, 17850)
(9, '536366', '22632', 6, Decimal('1.85'), 2, 17850)
(10, '536367', '84879', 32, Decimal('1.69'), 3, 13047)
(11, '536367', '22745', 6, Decimal('2.1'), 3, 13047)
(12, '536367', '22748', 6, Decimal('2.1'), 3, 13047)
(13, '536367', '22749', 8, Decimal('3.75'), 3, 13047)
(14, '536367', '22310', 6, Decimal('1.65'), 3, 13047)
(15, '536367', '84969', 6, Decimal('4.25'), 3, 13047)
(16, '536367', '22623', 3, Decimal('4.95'), 3, 13047)
(17, '536367', '22622', 2, Decimal('9.95'), 3, 13047)
(18, '536367', '21754', 3, Decimal('5.95'), 3, 13047)
(19, '536367', '21755', 3, Decimal